In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import os

In [2]:
cik_list = ['1824937', '1736388', '1811231', '1446275', '1796036', '1824627', '1421744', '1588504', '1627282', '1621653', '1826050', '1742055', '1818808']

In [3]:
# functions to scrape and parse filings


# returns pandas dataframe with filing information by cik
def cik_pull(cik):
    endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

    # parameters dictionary
    param_dict = {'action':'getcompany',
                  'CIK':cik,
                  'type':'',
                  'dateb':'',
                  'owner':'exclude',
                  'start':'',
                  'output':'atom',
                  'count':'100'}


    response = requests.get(url = endpoint, params = param_dict)
    soup = BeautifulSoup(response.content, 'lxml')

    entries = soup.find_all('entry')

    file_type_list = []
    file_date_list = []
    url_list = []
    acc_num_list = []

    for entry in entries:
        file_type_list.append(entry.find('filing-type').text)
        file_date_list.append(entry.find('filing-date').text)
        acc_num_list.append(entry.find('accession-number').text)
        url_list.append(entry.find('filing-href').text)
    

    filings_df = pd.DataFrame()
    filings_df['file_type'] = file_type_list
    filings_df['file_date'] = file_date_list
    filings_df['acc_num'] = acc_num_list
    filings_df['url'] = url_list


    #this is for simplicity dealing with urls
    temp_list = []
    for i in acc_num_list:
        temp_list.append(i.replace('-', ''))
    filings_df['acc_num_url'] = temp_list
    
    return filings_df

# returns dataframe of an individual filing as 1 row, and however many parsed fields as columns
def idv_document_parse(acc_num):
    root_url = 'https://www.sec.gov/Archives/edgar/data/'
    cik_num = cik
    acc_num_url = acc_num
    xml_end = 'primary_doc.xml'
    target_url = root_url + cik_num + '/' + acc_num_url + '/' + xml_end
    
    file = requests.get(target_url)
    file_data = BeautifulSoup(file.content, 'lxml')
    
    tag_list = []
    data_list = []
    for i in file_data.find_all():
        tag_list.append(i.name)
        data_list.append(i.string)
    file_df = pd.DataFrame(columns = tag_list)
    data_series = pd.Series(data_list, index = file_df.columns)
    file_df = file_df.append(data_series, ignore_index=True)
    return file_df

# given a cik, returns dictionary with key value pair of unique file type - dataframe of parsed documents of that type
def all_files_parse(cik):
    filings_df = cik_pull(cik)
    
    file_dict = {}
    for file in filings_df['file_type'].unique():
        target_df = pd.DataFrame()
        iter_df = filings_df[filings_df['file_type'] == f'{file}']
        #print(iter_df['file_type'].iloc[0])
    
        for i in range(len(iter_df)):
            sub = idv_document_parse(str(iter_df['acc_num_url'].iloc[i]))
            sub = sub.loc[:, ~sub.columns.duplicated()] #note that this will nuke the 50+ columns of jurisdiction offered.
            target_df = target_df.append(sub).fillna(np.nan)
        file_dict[f'{file}'] = target_df
    return file_dict

In [4]:
%%time

company_data_dict = {}

for cik in cik_list:
    iter_dict = all_files_parse(cik)
    company_data_dict[cik] = iter_dict
    print('-'*64)
    print(f'{cik}')
    for file in iter_dict.keys():
          print(f'#   {file}')
    

----------------------------------------------------------------
1824937
#   C
----------------------------------------------------------------
1736388
#   C/A
#   C-AR
#   C
#   D
#   C-U
#   C-AR/A
----------------------------------------------------------------
1811231
#   C/A
#   D
#   C
----------------------------------------------------------------
1446275
#   1-SA
#   253G2
#   QUALIF
#   CORRESP
#   UPLOAD
#   1-A/A
#   C-AR/A
#   1-A
#   C-AR
#   C-U
#   D
#   C
#   D/A
#   C/A
#   REGDEX
----------------------------------------------------------------
1796036
#   C/A
#   C-AR
#   C
----------------------------------------------------------------
1824627
#   C
----------------------------------------------------------------
1421744
#   253G2
#   QUALIF
#   CORRESP
#   1-A/A
#   UPLOAD
#   1-A
#   C-AR
#   D
#   C-U
#   C
#   D/A
#   REGDEX
----------------------------------------------------------------
1588504
#   1-A-W
#   1-SA
#   253G2
#   QUALIF
#   CORRESP
#   1-A/A
#  

In [20]:
iter_dict

{'C/A':    html  body  edgarsubmission  headerdata submissiontype  filerinfo  filer  \
 0   NaN   NaN              NaN         NaN            C/A        NaN    NaN   
 0   NaN   NaN              NaN         NaN            C/A        NaN    NaN   
 0   NaN   NaN              NaN         NaN            C/A        NaN    NaN   
 0   NaN   NaN              NaN         NaN            C/A        NaN    NaN   
 0   NaN   NaN              NaN         NaN            C/A        NaN    NaN   
 0   NaN   NaN              NaN         NaN            C/A        NaN    NaN   
 0   NaN   NaN              NaN         NaN            C/A        NaN    NaN   
 
    filercredentials    filercik  filerccc  ...             issuer  \
 0               NaN  0001736388  XXXXXXXX  ...  R3 Printing, Inc.   
 0               NaN  0001736388  XXXXXXXX  ...  R3 Printing, Inc.   
 0               NaN  0001736388  XXXXXXXX  ...  R3 Printing, Inc.   
 0               NaN  0001736388  XXXXXXXX  ...  R3 Printing, Inc.   
 

In [6]:
company_data_dict.keys()

dict_keys(['1824937', '1736388', '1811231', '1446275', '1796036', '1824627', '1421744', '1588504', '1627282', '1621653', '1826050', '1742055', '1818808'])

In [19]:

for i in company_data_dict.keys():
    path = 'data\pierce_csv_pull\\'
    os.mkdir(path + f'{i}')
    new_path = path + i
    iter_dict = company_data_dict[i]
    for x in iter_dict.keys():
        output_df = iter_dict[x].transpose()
        outfile = r'{}{}'.format(new_path, x)
        output_df.to_csv(outfile + '.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'data/pierce_csv_pull/1736388/C/A.csv'

In [25]:
output_df

,0,0,0,0,0,0,0
html,NaN,NaN,NaN,NaN,NaN,NaN,NaN
body,NaN,NaN,NaN,NaN,NaN,NaN,NaN
edgarsubmission,NaN,NaN,NaN,NaN,NaN,NaN,NaN
headerdata,NaN,NaN,NaN,NaN,NaN,NaN,NaN
submissiontype,C/A,C/A,C/A,C/A,C/A,C/A,C/A
...,...,...,...,...,...,...,...
persontitle,Director; Vice President; Head of Growth; Chie...,Director; Treasurer; Head of Product; Chief Ex...,Director; Vice President; Head of Growth; Chie...,Director; Vice President; Head of Growth; Chie...,Director,Director,Director
signaturedate,09-28-2020,08-11-2020,06-11-2020,05-01-2020,08-16-2019,05-21-2019,06-19-2018
com:street2,NaN,SUITE 200,NaN,NaN,NaN,NaN,NaN
crdnumber,NaN,NaN,NaN,NaN,283874,283874,283874
